# BDII -- Sesión 1 -- Procesado inicial de datos

Esta hoja muestra cómo procesar o curar un conjunto de datos para hacerlos más accesibles a la hora de introducirlos en bases de datos. Utilizaremos un conjunto de datos existente en Internet, que se descargará, se procesará y se convertirá en un formato universal como CSV o JSON. En particular se trabajará:

- La descarga de los datos
- Inspección, identificación del formato y posible procesado
- Generación de un formato fácilmente digerible por las BBDD, como CSV o JSON

Comenzaremos instalando los paquetes necesarios:

In [ ]:
!sudo apt-get update -qq

In [ ]:
!sudo apt-get install -y git p7zip

Importamos algunos paquetes estándar para la hoja

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib

%matplotlib inline
matplotlib.style.use('ggplot')

In [ ]:
RunningInCOLAB = 'google.colab' in str(get_ipython()) if hasattr(__builtins__,'__IPYTHON__') else False

## Datos de Stackoverflow

## Descarga de los datos

En este caso los datos están disponibles en un repositorio git. Se pueden descargar también de la Web, pero se van actualizando. En este caso los descargamos del repositorio git para que todos tengáis los mismos.

In [ ]:
!wget https://github.com/dsevilla/bd2-data/raw/main/es.stackoverflow/es.stackoverflow.7z.001
!wget https://github.com/dsevilla/bd2-data/raw/main/es.stackoverflow/es.stackoverflow.7z.002
!wget https://github.com/dsevilla/bd2-data/raw/main/es.stackoverflow/es.stackoverflow.7z.003

In [ ]:
!ls -l

In [ ]:
!7zr x es.stackoverflow.7z.001

In [ ]:
!ls -l

## Inspección y procesado

Podemos inspeccionar los ficheros `.xml` para ver su contenido. Son XML, sí, pero ¿con qué formato?

In [ ]:
!head Posts.xml

Aunque se puede procesar el formato XML, lo que podemos ver es que cada entrada es exactamente una línea que comienza por `<row`, y que contiene un conjunto de atributos en formato `atributo="valor"`. Si lo comprobamos, incluso no existirá ninguna comilla doble **dentro** de otra comilla doble, así que podemos extraer esos pares de forma facil.

In [ ]:
import re

def process_file(fname):
  with open(fname, "r") as f:
    for line in f:
      if "<row" in line:
        attr_dict = {}
        (_,attrs) = line.split("<row ",2)
        for m in re.finditer(r"(\w*?)=\"(.*?)\"", attrs):
          attr_dict[m.group(1)] = m.group(2)
        yield attr_dict

In [ ]:
first_row = next(process_file("Posts.xml"))

In [ ]:
first_row.keys()

Hay que extraer el conjunto de atributos para saber qué columnas tendrá nuestra tabla/CSV o archivo JSON. Recuérdese que las dos primeras filas del archivo XML tenían diferentes atributos. ¿Cómo se haría esto?

In [ ]:
def get_all_attrs(iterator):
  all_attrs = set()
  for row in iterator:
    all_attrs.update(row.keys())
  return all_attrs

all_attrs = get_all_attrs(process_file("Posts.xml"))

Como sabemos que el atributo `Id` va a ser la clave primaria, lo ponemos al principio. Además, generamos una lista, uno un conjunto, para que el orden sea conocido.

In [ ]:
all_attrs.remove('Id')
all_attrs = list(all_attrs)
all_attrs.insert(0,'Id')
all_attrs

## Escritura del formato CSV

El formato CSV está especificado en el estándar RFC 4180. https://www.ietf.org/rfc/rfc4180.txt. En general se puede utilizar la biblioteca `csv` de Python 3 y vamos a exportar una línea de cabecera con todos los campos. https://docs.python.org/3/library/csv.html.

Tendremos en cuenta que todas las filas tienen que tener las mismas columnas y en el mismo orden dado por `all_attrs`.

In [ ]:
import csv

def write_csv(destfile, all_attrs, iterator):
  with open(destfile, 'w') as wf:
    cw = csv.writer(wf)

    # Escribir la línea de cabecera
    cw.writerow(all_attrs)

    # Recorrer el iterador
    for row in iterator:
      cw.writerow(map(lambda att: row.get(att) or '', all_attrs))

In [ ]:
write_csv('Posts.csv', all_attrs, process_file('Posts.xml'))

In [ ]:
!head Posts.csv

## Conversión hacia JSON

https://www.json.org/json-en.html

In [ ]:
import json

def csv_to_json(fname_csv, fname_json, pk):
    data_dict = {}

    with open(fname_csv, "r") as f_csv:
        csv_reader = csv.DictReader(f_csv)

        for rows in csv_reader:
            key = rows[pk]
            data_dict[key] = rows

    with open(fname_json, 'w') as f_json:
        f_json.write(json.dumps(data_dict, indent = 4))

In [ ]:
fname_csv = 'Posts.csv'
fname_json = 'Posts.json'

csv_to_json(fname_csv, fname_json, 'Id')

In [ ]:
!head Posts.json

Si nos damos cuenta, tenemos el problema de que el valor Id está por duplicado.

Vamos a ver como eliminar columnas que no queramos tener.

In [ ]:
def csv_to_json2(fname_csv, fname_json, pk):
    data_dict = {}

    with open(fname_csv, "r") as f_csv:
        csv_reader = csv.DictReader(f_csv)

        for rows in csv_reader:
            key = rows[pk]

            # Borramos los campos que nos interesen.
            del rows[pk]

            data_dict[key] = rows

    with open(fname_json, 'w') as f_json:
        f_json.write(json.dumps(data_dict, indent = 4))

In [ ]:
fname_csv = 'Posts.csv'
fname_json = 'Posts2.json'

csv_to_json2(fname_csv, fname_json, 'Id')

In [ ]:
!head -n 100 Posts2.json

Al escribir en formato JSON se nos queda un fichero compacto que no podemos dividir.

## JSON Lines



https://jsonlines.org/


In [ ]:
def csv_to_jsonl(fname_csv, fname_jsonl):
    with open(fname_csv, 'r') as f_csv:
        csv_reader = csv.DictReader(f_csv)

        with open(fname_jsonl, 'w') as f_jsonl:
            for row in csv_reader:
                json_line = json.dumps(row)
                f_jsonl.write(json_line + '\n')

In [ ]:
csv_to_jsonl('Posts.csv', 'Posts.jsonl')

In [ ]:
!head Posts.jsonl